In [ ]:
!pip install langgraph langchain pinecone langchain_community langchain_groq pypdf langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [ ]:
#setting up the api keys
from google.colab import userdata
DeepSeekR1=userdata.get('groq_api_key')
PineCone=userdata.get('pine_cone')
google_api_key = userdata.get('GOOGLE_API_KEY')

In [ ]:
import os
os.environ['GROQ_API_KEY']=DeepSeekR1
os.environ['PINECONE_API_KEY']=PineCone
os.environ["GOOGLE_API_KEY"] = google_api_key

**## #in my case i will use deepseekr1 and i will use the database pinecone** **to store the embeddings**

In [ ]:
#import dependencies
import langchain
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.pinecone import Pinecone
from langchain.memory import ConversationBufferWindowMemory
import pinecone

In [ ]:
#setup the llms
from langchain_groq import ChatGroq
Model = ChatGroq(
    model = "deepseek-r1-distill-llama-70b",
    temperature=0,
    api_key = DeepSeekR1
)

In [ ]:
#setup the pinecone
from pinecone import Pinecone,ServerlessSpec
pc=Pinecone(api_key=PineCone)

index_name = "test-database0"

pc.create_index(
    name = index_name,
    dimension =  768,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
index = pc.Index(index_name) #TO connect the database

In [ ]:
#prepare to load the document
document = PyPDFLoader("/content/DeepSeek_R1.pdf")
data = document.load()

def TextSplitter(text):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    add_start_index = True,
  )
  splitter = text_splitter.split_documents(text)
  return splitter

text = TextSplitter(data)

In [ ]:
text

[Document(metadata={'source': '/content/DeepSeek_R1.pdf', 'page': 0, 'page_label': '1', 'start_index': 0}, page_content='DeepSeek-R1: Incentivizing Reasoning Capability in LLMs via\nReinforcement Learning\nDeepSeek-AI\nresearch@deepseek.com\nAbstract\nWe introduce our first-generation reasoning models, DeepSeek-R1-Zero and DeepSeek-R1.\nDeepSeek-R1-Zero, a model trained via large-scale reinforcement learning (RL) without super-\nvised fine-tuning (SFT) as a preliminary step, demonstrates remarkable reasoning capabilities.\nThrough RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and intriguing\nreasoning behaviors. However, it encounters challenges such as poor readability, and language\nmixing. To address these issues and further enhance reasoning performance, we introduce\nDeepSeek-R1, which incorporates multi-stage training and cold-start data before RL. DeepSeek-\nR1 achieves performance comparable to OpenAI-o1-1217 on reasoning tasks. To support the\nresearch communit

In [ ]:
#load embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
)

<ipython-input-15-0950ad0ace46>:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#Now stroes the embeddings into pinecone
from langchain_pinecone import PineconeVectorStore
vector_from_documents = PineconeVectorStore.from_documents(
    documents=text,
    embedding=embeddings,
    index_name=index_name,
)

In [ ]:
#test with query and only get top 2 ressults

retriver = vector_from_documents.as_retriever(search_kwargs={"k": 1})

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=Model,
    chain_type="stuff",
    retriever=retriver,
    # return_source_documents=True,
)

In [ ]:
query = "Benchmark of deepseek r1?"

In [ ]:
qa_chain.invoke(query)

{'query': 'Benchmark of deepseek r1?',
 'result': "<think>\nOkay, so the user is asking about the benchmark of DeepSeek-R1. Let me look through the provided context to find relevant information.\n\nFirst, I see that DeepSeek-R1 uses English for reasoning and responses, which might affect its performance in other languages. That's more about language support though, not directly about benchmarks.\n\nNext, there's a section on Prompting Engineering. It mentions that DeepSeek-R1 is sensitive to prompts and that few-shot prompting actually makes its performance worse. They recommend using a zero-shot setting for better results. This is important because it tells me that the model's performance can vary based on how it's prompted, which might influence benchmark results depending on the prompting method used.\n\nThen, under Software Engineering Tasks, it says that DeepSeek-R1 hasn't shown huge improvements over DeepSeek-V3. The reason given is the long evaluation times affecting the efficie